# Interactive graph

# NYC Average Daily Salary by Borough

This interactive map displays the average daily salary per borough in New York City for the years 2015 to 2023.

## Interactive Choropleth Map

<div id="map"></div>
<div id="slider-container">
  <input type="range" min="2015" max="2023" value="2015" id="year-slider" step="1">
  <p>Year: <span id="year-display">2015</span></p>
</div>

## Visualization

```{js, echo=FALSE}
// JavaScript code goes here
// You can directly write your JavaScript code here or reference an external .js file
// Assuming d3 and topojson libraries are included

// Load the GeoJSON data for NYC boroughs and the salary data
const boroughsGeoJSON = 'path_to_new-york-city-boroughs.geojson';
const salaryData = 'path_to_processed_salary_data.csv';

// Set dimensions for the map
const width = 960;
const height = 600;

// Create SVG element
const svg = d3.select("#map")
              .append("svg")
              .attr("width", width)
              .attr("height", height);

// Define projection and path
const projection = d3.geoMercator().center([-73.94, 40.70]).scale(50000).translate([width / 2, height / 2]);
const path = d3.geoPath().projection(projection);

// Load and process data
d3.json(boroughsGeoJSON).then(geoJSON => {
    d3.csv(salaryData).then(data => {
        // Process data here
        // ...

        // Create map
        svg.selectAll(".borough")
           .data(geoJSON.features)
           .enter()
           .append("path")
           .attr("class", "borough")
           .attr("d", path)
           .style("fill", d => colorScale(/* function to determine fill based on salary */))
           .on("mouseover", /* function to handle mouseover event */)
           .on("mouseout", /* function to handle mouseout event */);

        // Create a year to data mapping
        const salaryByYearAndBorough = {};
        data.forEach(d => {
            const year = d['Fiscal.Year'];
            const borough = d['Work.Location.Borough'];
            const salary = d['Daily.Salary'];
            if (!salaryByYearAndBorough[year]) {
                salaryByYearAndBorough[year] = {};
            }
            salaryByYearAndBorough[year][borough] = +salary;
        });

        // Function to update the map
        function updateMap(year) {
            svg.selectAll(".borough")
               .style("fill", d => {
                   const borough = d.properties.boro_name.toUpperCase();
                   const salary = salaryByYearAndBorough[year][borough];
                   return colorScale(salary); // Define your colorScale function
               });
        }

        // Initial map display
        updateMap('2015');

        // Slider interaction
        d3.select("#year-slider").on("input", function() {
            const selectedYear = this.value;
            d3.select("#year-display").text(selectedYear);
            updateMap(selectedYear);
        });

        // Hover effect
        svg.selectAll(".borough")
           .on("mouseover", function(event, d) {
               const borough = d.properties.boro_name.toUpperCase();
               const year = d3.select("#year-display").text();
               const salary = salaryByYearAndBorough[year][borough];
               // Show tooltip or update some display element with the salary
               // For example:
               // d3.select("#tooltip").text(`Salary: ${salary}`).style("visibility", "visible");
           })
           .on("mouseout", function() {
               // Hide tooltip or reset display element
               // For example:
               // d3.select("#tooltip").style("visibility", "hidden");
           });
    });
});